In [1]:
import numpy as np
from qutip import *

from qutip.qip.circuit import QubitCircuit, Gate
from qutip.qip.operations import gate_sequence_product, rx
from qutip.qip.device import *
from qutip.qip import pulse

#print(qutip.qip.device.__file__)

#import qip.ethprocessor as qip
%load_ext autoreload
%autoreload 2

In [44]:
# Number of qubits in the system
N = 2

# Input state
input_states = ["0"]*N

# A list of integers for the dimension of each composite system.
dims = [3]*N

# Representation of a quantum program/algorithm, maintaining a sequence of gates.
qc = QubitCircuit(N = N, input_states = input_states, dims = dims, reverse_states = False)

qc.add_gate("RX", 0, None, np.pi, r"-\pi")
qc.add_gate("RX", 0, None, np.pi, r"-\pi")

In [42]:
from qutip.qip.device import DispersiveCavityQED
from qutip.qip.device import ModelProcessor


d = DispersiveCavityQED(N=1)

d.load_circuit(qc);

#d.plot_pulses()

#d.run_state(init_state=basis(2, 0),qc=qc)

(1, 4)
(2, 4)
2


In [43]:
from qip.ethprocessor import ETHProcessor
import numpy as np

omega = 5.708390 * 1000

# Self-Kerr coefficient (anharmonicity) in MHz
alpha = - 261.081457

eth = ETHProcessor(N = 1, resonance_freq = omega, anharmonicity = alpha)

eth.load_circuit(qc);

eth.plot_pulses(title='Hej',dpi=300)

ValueError: The row number of coeffs must be same as the number of control pulses.

In [39]:
psi0 = basis(3,0)
result = eth.run_state(init_state=psi0, analytical=False)
final_state = result.states[-1].tidyup(1.0e-3)

In [40]:
omega = 5.708390 * 1000 * 2*np.pi

# Self-Kerr coefficient (anharmonicity) in MHz
alpha = - 261.081457 * 2*np.pi

# unitary matrix to into transmon frame
rotate_U = Qobj([[1,0,0],
                 [0,1,0],
                 [0,0,np.exp(1j*(alpha)*0.05)]])

#rotate_U = (1j*H_qubit*t_total).expm()
                 
# rotated transmon state 
final_state = rotate_U*final_state

# project onto qubit subspace
#qubit_state = (basis(2,0)*basis(3,0).dag() + basis(2,1)*basis(3,1).dag())*final_state
qubit_state = Qobj(final_state.data[0:2], dims = [[2],[1]])

# normalize qubit state
qubit_state_norm = qubit_state / qubit_state.norm()

# target state
target_state = basis(2,1)

# fidelity
fid = fidelity(target_state, qubit_state_norm)
print('Fidelity = %s %%' % round(fid*100,4)) # Should be close to 100 %

Fidelity = 70.1132 %
